## Merging of Per-Core Logs
### Can we assume the following about per-core behavior: 
#### If all cores of a running experiment do similar work, then their individual behaviors - as exposed by their per-core logs - are similar to each other, and hence, the behavior of all cores can be merged into an overall behavior of the full experimental run.

In [23]:
import os
import time
import pandas as pd
import numpy as np

In [24]:
import eigen_analysis

cols = eigen_analysis.LINUX_COLS
time_unit = eigen_analysis.TIME_CONVERSION_khz
joules_unit = eigen_analysis.JOULE_CONVERSION

In [25]:
# get all itrs explored for some (dvfs, qps) pair
def list_itrs(rdtsc_dirname):
    itrs = []
    for file in os.listdir(rdtsc_dirname):
        tags = file.split('_')
        itr = tags[1]
        itrs.append(itr)
    itrs = list(set(itrs))
    return itrs

In [26]:
global itrs

In [27]:
def print_err_log(dvfs, qps, itr, rapl='135', run='0', err_dir = 'err_logs/'):
    err_filename = 'err_log_' + run + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
    if os.path.exists(err_dir + err_filename):
        err_file = open(err_dir + err_filename, 'r')
        print(err_file.read())
    else:
        print('EMPTY ERROR LOG')

In [28]:
def handle_neg_diffs(df_diffs, df, core, err_file):
    
    tmp = df_diffs.copy()

    # isolating rows with negative diffs
    tmp_neg = tmp[(tmp['joules_diff'] < 0) | (tmp['instructions_diff'] < 0) | (tmp['cycles_diff'] < 0) \
                   | (tmp['ref_cycles_diff'] < 0) | (tmp['llc_miss_diff'] < 0) | (tmp['timestamp_diff'] < 0)]

    # re-computing diffs if possible; else dropping rows
    for i,j in tmp_neg.iterrows():
        prev = df.shift(1).loc[i]
        cur = df.loc[i]
        # TODO note that we are only handling case of RAPL-energy-status register overflow
        if (tmp.loc[i]['joules_diff'] < 0) & (tmp.loc[i]['timestamp_diff'] >= 0.001):
            err_file.write('CORE ' + str(core) + '  ---  JOULES COUNTER OVERFLOW AT LOG ENTRY DIFF #' + str(i) + '\n')
            tmp.loc[i, ['joules_diff']] = (2**32 - 1) * joules_unit - prev['joules'] + cur['joules'] 
        else:
            err_file.write('CORE ' + str(core) + '  ---  UNEXPLAINED NEGATIVE VALS AT LOG ENTRY DIFF # ' + str(i) + '\n')
            cols = ''
            prevs = ''
            currs = ''
            for col in list(df.columns):
                cols += col + '  '
                prevs += str(prev[col]) + '  '
                currs += str(cur[col]) + '  '
            err_file.write('          ' + cols + '\n')
            err_file.write('         log[' + str(i-1) + ']: ' + prevs + '\n')
            err_file.write('         log[' + str(i) + ']: ' + currs + '\n')
            tmp = tmp.drop(i, axis=0)

    return tmp

In [29]:
# given (dvfs, qps, itr), concatenate all per-core logs into one big dataframe
def concat_core_logs(dvfs, qps, itr, rapl='135', run='0'):
    print('Concatenating all per-core logs with ITR-DELAY = ', itr)
    
    # here are all the log files for this dvfs & qps
    logs_dir = qps + '_qps/linux_mcd_dmesg_' + run + '_' + dvfs + '_' + rapl + '_' + qps + '/'
    # here are all the time-management files for this dvfs & qps
    rdtsc_dir = qps + '_qps/linux_mcd_rdtsc_' + run + '_' + dvfs + '_' + rapl + '_' + qps +'/'
    rdtsc_file = rdtsc_dir + 'linux.mcd.rdtsc.' + run + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
    start, end = eigen_analysis.get_rdtsc(rdtsc_file)
        
    # initializing error log file
    err_dir = 'err_logs/'
    err_filename = 'err_log_' + run + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
    err_file = open(err_dir + err_filename, 'w')

    # here will be stored counter-based log data from all cores
    counters_full_df = pd.DataFrame()
    # here will be stored non-counter-based log data from all cores
    non_counters_full_df = pd.DataFrame()
    
    # TODO remove fixed core-id range
    for c in range(0,16):
        file = logs_dir + 'linux.mcd.dmesg.' + run + '_' + str(c) + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
        df = pd.read_csv(file, sep = ' ', names = cols, index_col='i')
        df = df[(df['timestamp'] >= start) & (df['timestamp'] <= end)]
        df['timestamp'] = df['timestamp'] - df['timestamp'].min()
        df['timestamp'] = df['timestamp'] * time_unit
        df['joules'] = df['joules'] * joules_unit

        # CONCATENATING MILLISECOND-LEVEL PER-CORE DFS
        ##############################################
        # removing zero-filled log-entries
        # -> these represent interrupt occurrences at a frequency greater than per-1ms
        counters_df = df[['joules', 'instructions', 'cycles', 'ref_cycles', 'llc_miss', 'timestamp']].copy()
        counters_df = counters_df[(counters_df['joules'] > 0) & (counters_df['instructions'] > 0) \
                                            & (counters_df['cycles'] > 0) & (counters_df['ref_cycles'] > 0) \
                                            & (counters_df['llc_miss'] > 0)]

        tmp = counters_df['timestamp']        
        # computing diffs of counter readings
        df_diffs = counters_df.diff().dropna().copy()
        df_diffs.columns = [f'{c}_diff' for c in df_diffs.columns]
        df_diffs = handle_neg_diffs(df_diffs, counters_df, c, err_file)
        df_diffs = df_diffs.drop(['timestamp_diff'], axis=1)
        df_diffs['timestamp'] = tmp
        
        if counters_full_df.shape[0] == 0:
            counters_full_df = df_diffs.copy()
        else:
            counters_full_df = counters_full_df.merge(df_diffs, \
                                                      left_on = 'timestamp', \
                                                      right_on = 'timestamp', \
                                                      how='outer', \
                                                      sort=True, \
                                                      suffixes=('', '_0')).fillna(0)        

        # CONCATENATING MICROSECOND-LEVEL PER-CORE DFS
        ##############################################
        non_counters_df = df[['rx_bytes', 'rx_desc', 'tx_bytes', 'tx_desc', 'timestamp']].copy()                                                          

        if non_counters_full_df.shape[0] == 0:
            non_counters_full_df = non_counters_df.copy()
        else:
            non_counters_full_df = non_counters_full_df.merge(non_counters_df, \
                                                              left_on = 'timestamp', \
                                                              right_on = 'timestamp', \
                                                              how='outer', \
                                                              sort=True, \
                                                              suffixes=('', '_0')).fillna(0)         
            
        print('CORE: ', str(c))
        print('         NON COUNTERS:  full =', non_counters_df.shape[0], \
              '  expected:', int(20 * 10**6 / int(itr)))        
        print('         COUNTERS:      full =', counters_df.shape[0], \
              '  after computing diffs =', df_diffs.shape[0]          \
             )

    # delete error log if empty
    err_file.close()
    if (os.path.getsize(err_dir + err_filename) == 0):
        os.remove(err_dir + err_filename)
        
    print()
    print('-------------------------------------------------- PARSED 16 LOGS -------------------------')
    print()
    return counters_full_df, non_counters_full_df

In [30]:
def merge_concat_logs(counters_full_df, non_counters_full_df):
    
    # creating dfs of average per-core log readings
    df_merged_counter = pd.DataFrame(columns=['instructions_diff', 'cycles_diff', 'ref_cycles_diff', \
                                     'llc_miss_diff', 'joules_diff'])
    df_merged_non_counter = pd.DataFrame(columns=['rx_bytes', 'rx_desc', 'tx_bytes', 'tx_desc'])

    for col in df_merged_counter.columns:
        df_merged_counter[col] = (counters_full_df[[col, col+'_0']].sum(axis=1))
    df_merged_counter['timestamp'] = counters_full_df['timestamp']
    for col in df_merged_non_counter.columns:
        df_merged_non_counter[col] = (non_counters_full_df[[col, col+'_0']].sum(axis=1))
    df_merged_non_counter['timestamp'] = non_counters_full_df['timestamp']
    
    return df_merged_counter, df_merged_non_counter

In [31]:
def save_merged_logs(merged_counters_df, merged_non_counters_df, dvfs, qps, itr, rapl='135', run='0'):
    
    counters_df_outdir = qps + '_' + dvfs + '_counters_merged/'
    non_counters_df_outdir = qps + '_' + dvfs + '_non_counters_merged/'
    !mkdir -p $counters_df_outdir
    !mkdir -p $non_counters_df_outdir
    counters_outfile = counters_df_outdir + dvfs + '_' + qps + '_' + itr + '_counters_merged'
    non_counters_outfile = non_counters_df_outdir + dvfs + '_' + qps + '_' + itr + '_non_counters_merged'
    merged_counters_df.to_csv(counters_outfile)
    merged_non_counters_df.to_csv(non_counters_outfile)

    print('COUNTERS DIR: ')
    !ls $counters_df_outdir
    print('NON_COUNTERS DIR: ')
    !ls $non_counters_df_outdir

In [32]:
def concat_merge_core_logs(dvfs, qps, itr, rapl='135', run='0'):
    print(f'Concatenating and merging all per-core logs with DVFS={dvfs} ITR-DELAY={itr}, QPS={qps}')
    
    logs_dir = qps + '_qps/linux_mcd_dmesg_' + run + '_' + dvfs + '_' + rapl + '_' + qps + '/'
    rdtsc_dir = qps + '_qps/linux_mcd_rdtsc_' + run + '_' + dvfs + '_' + rapl + '_' + qps +'/'
    rdtsc_file = rdtsc_dir + 'linux.mcd.rdtsc.' + run + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
    start, end = eigen_analysis.get_rdtsc(rdtsc_file)
    err_dir = 'err_logs/'
    err_filename = 'err_log_' + run + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
    err_file = open(err_dir + err_filename, 'w')

    counters_full_df = pd.DataFrame()
    non_counters_full_df = pd.DataFrame()    
    
    for c in range(0,16):
        df_merged_counters = pd.DataFrame(columns=['instructions_diff', 'cycles_diff', 'ref_cycles_diff', \
                                     'llc_miss_diff', 'joules_diff', 'timestamp'])
        df_merged_non_counters = pd.DataFrame(columns=['rx_bytes', 'rx_desc', 'tx_bytes', 'tx_desc', 'timestamp'])
        
        file = logs_dir + 'linux.mcd.dmesg.' + run + '_' + str(c) + '_' + itr + '_' + dvfs + '_' + rapl + '_' + qps[:-1] + '000'
        df = pd.read_csv(file, sep = ' ', names = cols, index_col='i')
        df = df[(df['timestamp'] >= start) & (df['timestamp'] <= end)]
        #df['timestamp'] = df['timestamp'] - df['timestamp'].min()
        df['timestamp'] = df['timestamp'] * time_unit
        df['joules'] = df['joules'] * joules_unit

        counters_df = df[['joules', 'instructions', 'cycles', 'ref_cycles', 'llc_miss', 'timestamp']].copy()
        counters_df = counters_df[(counters_df['joules'] > 0) & (counters_df['instructions'] > 0) \
                                            & (counters_df['cycles'] > 0) & (counters_df['ref_cycles'] > 0) \
                                            & (counters_df['llc_miss'] > 0)]
        tmp = counters_df['timestamp']        
        df_diffs = counters_df.diff().dropna().copy()
        df_diffs.columns = [f'{c}_diff' for c in df_diffs.columns]
        df_diffs = handle_neg_diffs(df_diffs, counters_df, c, err_file)
        df_diffs = df_diffs.drop(['timestamp_diff'], axis=1)
        df_diffs['timestamp'] = tmp        
        if counters_full_df.shape[0] == 0:
            counters_full_df = df_diffs.copy()
        else:
            counters_full_df = counters_full_df.merge(df_diffs, \
                                                      left_on = 'timestamp', \
                                                      right_on = 'timestamp', \
                                                      how='outer', \
                                                      sort=True, \
                                                      suffixes=('', '_0')).fillna(0) 
            for col in df_merged_counters.columns:
                if col == 'timestamp':
                    break
                else:
                    df_merged_counters[col] = (counters_full_df[[col, col+'_0']].sum(axis=1))    
            df_merged_counters['timestamp'] = counters_full_df['timestamp']
            counters_full_df = df_merged_counters.copy()

        non_counters_df = df[['rx_bytes', 'rx_desc', 'tx_bytes', 'tx_desc', 'timestamp']].copy()                                                          
        if non_counters_full_df.shape[0] == 0:
            non_counters_full_df = non_counters_df.copy()
        else:
            non_counters_full_df = non_counters_full_df.merge(non_counters_df, \
                                                              left_on = 'timestamp', \
                                                              right_on = 'timestamp', \
                                                              how='outer', \
                                                              sort=True, \
                                                              suffixes=('', '_0')).fillna(0) 
            for col in df_merged_non_counters.columns:
                if col == 'timestamp':
                    break
                else:
                    df_merged_non_counters[col] = (non_counters_full_df[[col, col+'_0']].sum(axis=1))
            df_merged_non_counters['timestamp'] = non_counters_full_df['timestamp']
            non_counters_full_df = df_merged_non_counters.copy()
                
        print('CORE: ', str(c))
        print('         COUNTERS:      full =', counters_df.shape[0], \
              '  after computing diffs =', df_diffs.shape[0])
        print('         MERGED COUNTERS: ', df_merged_counters.shape)
        print()
        print('         NON COUNTERS:  full =', non_counters_df.shape[0], \
              '  expected:', int(20 * 10**6 / int(itr)))  
        print('         MERGED NON COUNTERS: ', df_merged_non_counters.shape)

    err_file.close()
    if (os.path.getsize(err_dir + err_filename) == 0):
        os.remove(err_dir + err_filename)
        
    print()
    print('-------------------------------------------------- PARSED 16 LOGS -------------------------')
    print()
    return counters_full_df, non_counters_full_df, df_merged_counters, df_merged_non_counters    

In [41]:
dvfs = '0xf00'
for qps in ['400k', '600k']:
    #qps = '400k'
    run = '0'
    rapl = '135'
    itrs = list_itrs(qps + '_qps/linux_mcd_rdtsc_' + run + '_' + dvfs + '_' + rapl + '_' + qps +'/')
    for itr in ['100', '20', '250', '30', '350', '40', '10', '200', '300', '50', '400', '2']:
    #for itr in ['100', '20', '250', '30', '350', '40']:
        counters_full_df = pd.DataFrame()
        non_counters_full_df = pd.DataFrame()

        #counters_full_df, non_counters_full_df = concat_core_logs(dvfs, qps, itr)
        counters_full_df, non_counters_full_df, merged_counters_df, merged_non_counters_df = concat_merge_core_logs(dvfs, qps, itr)

        counters_full_df['timestamp'] = counters_full_df['timestamp'] - counters_full_df['timestamp'].min()
        non_counters_full_df['timestamp'] = non_counters_full_df['timestamp'] - non_counters_full_df['timestamp'].min()
        merged_counters_df['timestamp'] = merged_counters_df['timestamp'] - merged_counters_df['timestamp'].min()
        merged_non_counters_df['timestamp'] = merged_non_counters_df['timestamp'] - merged_non_counters_df['timestamp'].min()

        print_err_log(dvfs, qps, itr)
        #merged_counters_df, merged_non_counters_df = merge_concat_logs(counters_full_df, non_counters_full_df)
        save_merged_logs(merged_counters_df, merged_non_counters_df, dvfs, qps, itr)
        print('----------------------------------------')

Concatenating and merging all per-core logs with DVFS=0xf00 ITR-DELAY=100, QPS=400k
CORE:  0
         COUNTERS:      full = 19496   after computing diffs = 19495
         MERGED COUNTERS:  (0, 6)

         NON COUNTERS:  full = 194770   expected: 200000
         MERGED NON COUNTERS:  (0, 5)
CORE:  1
         COUNTERS:      full = 19494   after computing diffs = 19493
         MERGED COUNTERS:  (38988, 6)

         NON COUNTERS:  full = 194670   expected: 200000
         MERGED NON COUNTERS:  (389440, 5)
CORE:  2
         COUNTERS:      full = 19493   after computing diffs = 19492
         MERGED COUNTERS:  (58480, 6)

         NON COUNTERS:  full = 194855   expected: 200000
         MERGED NON COUNTERS:  (584292, 5)
CORE:  3
         COUNTERS:      full = 19492   after computing diffs = 19491
         MERGED COUNTERS:  (77971, 6)

         NON COUNTERS:  full = 194791   expected: 200000
         MERGED NON COUNTERS:  (779080, 5)
CORE:  4
         COUNTERS:      full = 19491   after com

CORE:  2
         COUNTERS:      full = 19536   after computing diffs = 19535
         MERGED COUNTERS:  (58603, 6)

         NON COUNTERS:  full = 78141   expected: 80000
         MERGED NON COUNTERS:  (234421, 5)
CORE:  3
         COUNTERS:      full = 19535   after computing diffs = 19534
         MERGED COUNTERS:  (78137, 6)

         NON COUNTERS:  full = 78138   expected: 80000
         MERGED NON COUNTERS:  (312554, 5)
CORE:  4
         COUNTERS:      full = 19535   after computing diffs = 19534
         MERGED COUNTERS:  (97671, 6)

         NON COUNTERS:  full = 78140   expected: 80000
         MERGED NON COUNTERS:  (390694, 5)
CORE:  5
         COUNTERS:      full = 19535   after computing diffs = 19534
         MERGED COUNTERS:  (117205, 6)

         NON COUNTERS:  full = 78138   expected: 80000
         MERGED NON COUNTERS:  (468832, 5)
CORE:  6
         COUNTERS:      full = 19536   after computing diffs = 19535
         MERGED COUNTERS:  (136740, 6)

         NON COUNTERS

CORE:  2
         COUNTERS:      full = 18604   after computing diffs = 18603
         MERGED COUNTERS:  (55811, 6)

         NON COUNTERS:  full = 55814   expected: 57142
         MERGED NON COUNTERS:  (167443, 5)
CORE:  3
         COUNTERS:      full = 18605   after computing diffs = 18604
         MERGED COUNTERS:  (74415, 6)

         NON COUNTERS:  full = 55814   expected: 57142
         MERGED NON COUNTERS:  (223257, 5)
CORE:  4
         COUNTERS:      full = 18605   after computing diffs = 18604
         MERGED COUNTERS:  (93019, 6)

         NON COUNTERS:  full = 55814   expected: 57142
         MERGED NON COUNTERS:  (279071, 5)
CORE:  5
         COUNTERS:      full = 18604   after computing diffs = 18603
         MERGED COUNTERS:  (111622, 6)

         NON COUNTERS:  full = 55814   expected: 57142
         MERGED NON COUNTERS:  (334885, 5)
CORE:  6
         COUNTERS:      full = 18605   after computing diffs = 18604
         MERGED COUNTERS:  (130226, 6)

         NON COUNTERS

CORE:  2
         COUNTERS:      full = 19659   after computing diffs = 19658
         MERGED COUNTERS:  (58987, 6)

         NON COUNTERS:  full = 875031   expected: 2000000
         MERGED NON COUNTERS:  (2624021, 5)
CORE:  3
         COUNTERS:      full = 19674   after computing diffs = 19673
         MERGED COUNTERS:  (78660, 6)

         NON COUNTERS:  full = 880411   expected: 2000000
         MERGED NON COUNTERS:  (3504344, 5)
CORE:  4
         COUNTERS:      full = 19661   after computing diffs = 19660
         MERGED COUNTERS:  (98320, 6)

         NON COUNTERS:  full = 869518   expected: 2000000
         MERGED NON COUNTERS:  (4373743, 5)
CORE:  5
         COUNTERS:      full = 19667   after computing diffs = 19666
         MERGED COUNTERS:  (117986, 6)

         NON COUNTERS:  full = 875470   expected: 2000000
         MERGED NON COUNTERS:  (5249185, 5)
CORE:  6
         COUNTERS:      full = 19660   after computing diffs = 19659
         MERGED COUNTERS:  (137645, 6)

     

----------------------------------------
Concatenating and merging all per-core logs with DVFS=0xf00 ITR-DELAY=300, QPS=400k
CORE:  0
         COUNTERS:      full = 16279   after computing diffs = 16278
         MERGED COUNTERS:  (0, 6)

         NON COUNTERS:  full = 65108   expected: 66666
         MERGED NON COUNTERS:  (0, 5)
CORE:  1
         COUNTERS:      full = 16280   after computing diffs = 16279
         MERGED COUNTERS:  (32557, 6)

         NON COUNTERS:  full = 65117   expected: 66666
         MERGED NON COUNTERS:  (130224, 5)
CORE:  2
         COUNTERS:      full = 16280   after computing diffs = 16279
         MERGED COUNTERS:  (48836, 6)

         NON COUNTERS:  full = 65118   expected: 66666
         MERGED NON COUNTERS:  (195342, 5)
CORE:  3
         COUNTERS:      full = 16280   after computing diffs = 16279
         MERGED COUNTERS:  (65115, 6)

         NON COUNTERS:  full = 65118   expected: 66666
         MERGED NON COUNTERS:  (260459, 5)
CORE:  4
         COUNTE

...

NON_COUNTERS DIR: 
0xf00_600k_100_non_counters_merged  0xf00_600k_250_non_counters_merged
0xf00_600k_20_non_counters_merged   0xf00_600k_30_non_counters_merged
----------------------------------------
Concatenating and merging all per-core logs with DVFS=0xf00 ITR-DELAY=350, QPS=600k
CORE:  0
         COUNTERS:      full = 18605   after computing diffs = 18604
         MERGED COUNTERS:  (0, 6)

         NON COUNTERS:  full = 55814   expected: 57142
         MERGED NON COUNTERS:  (0, 5)
CORE:  1
         COUNTERS:      full = 18605   after computing diffs = 18604
         MERGED COUNTERS:  (37208, 6)

         NON COUNTERS:  full = 55815   expected: 57142
         MERGED NON COUNTERS:  (111629, 5)
CORE:  2
         COUNTERS:      full = 18605   after computing diffs = 18604
         MERGED COUNTERS:  (55812, 6)

         NON COUNTERS:  full = 55815   expected: 57142
         MERGED NON COUNTERS:  (167444, 5)
CORE:  3
         COUNTERS:      full = 18605   after computing diffs = 18604
 

----------------------------------------
Concatenating and merging all per-core logs with DVFS=0xf00 ITR-DELAY=10, QPS=600k
CORE:  0
         COUNTERS:      full = 19771   after computing diffs = 19770
         MERGED COUNTERS:  (0, 6)

         NON COUNTERS:  full = 1170642   expected: 2000000
         MERGED NON COUNTERS:  (0, 5)
CORE:  1
         COUNTERS:      full = 19779   after computing diffs = 19778
         MERGED COUNTERS:  (39548, 6)

         NON COUNTERS:  full = 1187491   expected: 2000000
         MERGED NON COUNTERS:  (2358109, 5)
CORE:  2
         COUNTERS:      full = 19777   after computing diffs = 19776
         MERGED COUNTERS:  (59324, 6)

         NON COUNTERS:  full = 1177578   expected: 2000000
         MERGED NON COUNTERS:  (3535549, 5)
CORE:  3
         COUNTERS:      full = 19781   after computing diffs = 19780
         MERGED COUNTERS:  (79104, 6)

         NON COUNTERS:  full = 1172537   expected: 2000000
         MERGED NON COUNTERS:  (4707996, 5)
CORE: 

NON_COUNTERS DIR: 
0xf00_600k_100_non_counters_merged  0xf00_600k_250_non_counters_merged
0xf00_600k_10_non_counters_merged   0xf00_600k_30_non_counters_merged
0xf00_600k_200_non_counters_merged  0xf00_600k_350_non_counters_merged
0xf00_600k_20_non_counters_merged   0xf00_600k_40_non_counters_merged
----------------------------------------
Concatenating and merging all per-core logs with DVFS=0xf00 ITR-DELAY=300, QPS=600k
CORE:  0
         COUNTERS:      full = 16280   after computing diffs = 16279
         MERGED COUNTERS:  (0, 6)

         NON COUNTERS:  full = 65118   expected: 66666
         MERGED NON COUNTERS:  (0, 5)
CORE:  1
         COUNTERS:      full = 16279   after computing diffs = 16278
         MERGED COUNTERS:  (32557, 6)

         NON COUNTERS:  full = 65115   expected: 66666
         MERGED NON COUNTERS:  (130233, 5)
CORE:  2
         COUNTERS:      full = 16280   after computing diffs = 16279
         MERGED COUNTERS:  (48836, 6)

         NON COUNTERS:  full = 65118

COUNTERS DIR: 
0xf00_600k_100_counters_merged	0xf00_600k_300_counters_merged
0xf00_600k_10_counters_merged	0xf00_600k_30_counters_merged
0xf00_600k_200_counters_merged	0xf00_600k_350_counters_merged
0xf00_600k_20_counters_merged	0xf00_600k_40_counters_merged
0xf00_600k_250_counters_merged	0xf00_600k_50_counters_merged
NON_COUNTERS DIR: 
0xf00_600k_100_non_counters_merged  0xf00_600k_300_non_counters_merged
0xf00_600k_10_non_counters_merged   0xf00_600k_30_non_counters_merged
0xf00_600k_200_non_counters_merged  0xf00_600k_350_non_counters_merged
0xf00_600k_20_non_counters_merged   0xf00_600k_40_non_counters_merged
0xf00_600k_250_non_counters_merged  0xf00_600k_50_non_counters_merged
----------------------------------------
Concatenating and merging all per-core logs with DVFS=0xf00 ITR-DELAY=400, QPS=600k
CORE:  0
         COUNTERS:      full = 16279   after computing diffs = 16278
         MERGED COUNTERS:  (0, 6)

         NON COUNTERS:  full = 48838   expected: 50000
         MERGED 

CORE:  14
         COUNTERS:      full = 19820   after computing diffs = 19819
         MERGED COUNTERS:  (297162, 6)

         NON COUNTERS:  full = 1817793   expected: 10000000
         MERGED NON COUNTERS:  (26961026, 5)
CORE:  15
         COUNTERS:      full = 19806   after computing diffs = 19805
         MERGED COUNTERS:  (316967, 6)

         NON COUNTERS:  full = 1779601   expected: 10000000
         MERGED NON COUNTERS:  (28739673, 5)

-------------------------------------------------- PARSED 16 LOGS -------------------------

EMPTY ERROR LOG
COUNTERS DIR: 
0xf00_600k_100_counters_merged	0xf00_600k_300_counters_merged
0xf00_600k_10_counters_merged	0xf00_600k_30_counters_merged
0xf00_600k_200_counters_merged	0xf00_600k_350_counters_merged
0xf00_600k_20_counters_merged	0xf00_600k_400_counters_merged
0xf00_600k_250_counters_merged	0xf00_600k_40_counters_merged
0xf00_600k_2_counters_merged	0xf00_600k_50_counters_merged
NON_COUNTERS DIR: 
0xf00_600k_100_non_counters_merged  0xf00_6

In [19]:
counters_full_df

,instructions_diff,cycles_diff,ref_cycles_diff,llc_miss_diff,joules_diff,timestamp
0,889222.0,2123830.0,2595036.0,5182.0,0.109678,0.000000
1,1120698.0,2402735.0,2884514.0,7504.0,0.109678,0.000007
2,1047079.0,2407312.0,2881121.0,6923.0,0.191662,0.000024
3,1024170.0,2374815.0,2799979.0,6821.0,0.191662,0.000026
4,1124788.0,2401682.0,2833822.0,6816.0,0.191662,0.000032
...,...,...,...,...,...,...
312422,343621.0,880492.0,1021380.0,1865.0,0.206790,20.003250
312423,697432.0,1713130.0,1987254.0,3801.0,0.225517,20.003272
312424,519100.0,1350211.0,1566232.0,3181.0,0.225517,20.003274
312425,488304.0,1146019.0,1329360.0,2497.0,0.206790,20.003277


In [20]:
non_counters_full_df

,rx_bytes,rx_desc,tx_bytes,tx_desc,timestamp
0,132.0,2.0,0.0,0.0,0.000000
1,66.0,1.0,0.0,0.0,0.000008
2,66.0,1.0,0.0,0.0,0.000022
3,66.0,1.0,0.0,0.0,0.000022
4,132.0,2.0,74.0,2.0,0.000023
...,...,...,...,...,...
7654793,132.0,2.0,66.0,2.0,20.004338
7654794,132.0,2.0,66.0,2.0,20.004340
7654795,132.0,2.0,132.0,4.0,20.004341
7654796,132.0,2.0,132.0,4.0,20.004341


In [21]:
merged_counters_df

,instructions_diff,cycles_diff,ref_cycles_diff,llc_miss_diff,joules_diff,timestamp
0,889222.0,2123830.0,2595036.0,5182.0,0.109678,0.000000
1,1120698.0,2402735.0,2884514.0,7504.0,0.109678,0.000007
2,1047079.0,2407312.0,2881121.0,6923.0,0.191662,0.000024
3,1024170.0,2374815.0,2799979.0,6821.0,0.191662,0.000026
4,1124788.0,2401682.0,2833822.0,6816.0,0.191662,0.000032
...,...,...,...,...,...,...
312422,343621.0,880492.0,1021380.0,1865.0,0.206790,20.003250
312423,697432.0,1713130.0,1987254.0,3801.0,0.225517,20.003272
312424,519100.0,1350211.0,1566232.0,3181.0,0.225517,20.003274
312425,488304.0,1146019.0,1329360.0,2497.0,0.206790,20.003277


In [22]:
merged_non_counters_df

,rx_bytes,rx_desc,tx_bytes,tx_desc,timestamp
0,132.0,2.0,0.0,0.0,0.000000
1,66.0,1.0,0.0,0.0,0.000008
2,66.0,1.0,0.0,0.0,0.000022
3,66.0,1.0,0.0,0.0,0.000022
4,132.0,2.0,74.0,2.0,0.000023
...,...,...,...,...,...
7654793,132.0,2.0,66.0,2.0,20.004338
7654794,132.0,2.0,66.0,2.0,20.004340
7654795,132.0,2.0,132.0,4.0,20.004341
7654796,132.0,2.0,132.0,4.0,20.004341
